# Deep Reinforcement Learning

 Reinforcement Learning (RL) is an approach wherein an agent learns to make sequential decisions by interacting with an environment. The objective is for the agent to maximize the cumulative reward it receives over time.
 The agent goes through this process by repeatedly evaluating the consequences of its actions, trying to select actions that lead to better outcomes.

To do this, we will use Gym, an platform for developing and comparing reinforcement learning algorithms. Gym provides an interface for interacting with different environments, it accepts actions from agents and plays them out in an environment, providing rewards.


## Environment

We will be using `CartPole` environment from gym's library for this assignment.  In this environment, a pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The pendulum is placed upright on the cart and the goal is to balance the pole by applying forces in the left and right direction on the cart.

You can use the code below to run an instance of a random agent in this environment and see the results.

In [ ]:
from IPython.display import HTML
from base64 import b64encode

def show_video(path):
    mp4 = open(path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=400 controls>
          <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
!pip install gym[atari,accept-rom-license] -qq
!pip install imageio -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 30.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import cv2
import gym
import imageio
import numpy as np
from gym import spaces

We use `gym.make()` to make an instance of a certain environemtn. We can then use `.step()` method which accepts an action as input and performs it. Before that we reset the environment to its initial state by using `.reset()` method.

In [ ]:
env_name = 'CartPole-v1'

env = gym.make(env_name)

env.reset()

frames = []

for _ in range(500):
    action = env.action_space.sample()

    obs, reward, done, _ = env.step(action)


    frames.append(env.render(mode="rgb_array"))

    if done:
        env.reset()

env.close()
imageio.mimsave('./cartpole.mp4', frames, fps=25)



/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/usr/local/lib/python3.10/dist-packages/gym/core.py:43: DeprecationWarning: WARN: The argument mode in render method is deprecated; use render_mode during environment initiali

In [ ]:
show_video('./cartpole.mp4')

As you can see, the cart fails to keep the balance of the pole. In the next section we will train an agent to learn how to perform this task.

## Algorithm
We will be using A2C algorithm.

Advantage Actor-Critic (A2C) is a reinforcement learning algorithm.
It consists of an actor (which predicts the best action based on the current state) and a critic (which estimates the state's value function to measure expected future rewards).

We will implement this together step by step.




In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions.categorical import Categorical

import numpy as np
import gym
from collections import deque
from tqdm import tqdm

## Neural Network

Here we design a simple feed forward model to embed the observation from the environment to a hidden layer. We then use two fully connected layers on top of the hidden layer, to predict the next action and estimate the value of current state. This acts as both actor, and critic.


In [5]:
class ActorCritic(nn.Module):
    def __init__(self, hidden_size, num_outputs):
        super(ActorCritic, self).__init__()
        self.fc1 = nn.Linear(4, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)

        # Actor
        self.action_head = nn.Linear(hidden_size, num_outputs)

        # Critic
        self.value_head = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        action_probs = F.softmax(self.action_head(x), dim=-1)
        value = self.value_head(x)

        return action_probs, value


## A2C

The A2C algorithm aims to jointly train both the actor and the critic to improve the policy. It does this by updating the parameters
of the actor to increase the likelihood of good actions and updating the parameters
of the critic to better estimate the value function.

In each iteration A2C plays the until it ends. During this time it records log probabality of actions, rewards, and predicted values in each step. These values will be used to update the model at the end of this trajectory.

The actor is updated using the objective below:

$$ L_{\text{actor}} = -\log \pi(a|s;\theta) \times A(s, a) $$
Where advantage is calculated as:
$$A(s, a) = Q(s, a) - V(s) $$

Namely the function $Q(s,a)$ is the estimated value of taking action
$a$
 in state
$s$.
$V(s)$ is the predicted value of our critic.

This loss function aims to improve the probability of playing actions that result in higher rewards.

As for the critic the loss function is defined as a simple mean square loss between actual value of an state and the predicted one:

$$ L_{\text{critic}} = \frac{1}{2} ( R - V(s))^2 $$

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions.categorical import Categorical
import numpy as np
import gym
from collections import deque
from tqdm import tqdm

class A2CAgent:
    def __init__(self, env, num_episodes=1000, max_steps=500, gamma=0.99, lr=1e-3, hidden_size=256):
        self.env = env
        self.num_episodes = int(num_episodes)
        self.max_steps = max_steps
        self.gamma = gamma
        self.lr = lr
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.policy_net = ActorCritic(hidden_size, env.action_space.n).to(self.device)
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=self.lr)
        self.critic_loss = nn.MSELoss()

    def choose_action(self, state):
        if not isinstance(state, np.ndarray):
            state = np.array(state)
        state = torch.FloatTensor(state).to(self.device)
        action_probs, _ = self.policy_net(state)
        dist = Categorical(action_probs)
        action = dist.sample()
        return action.item(), dist.log_prob(action)

    def compute_returns(self, rewards):
        returns = []
        R = 0
        for r in reversed(rewards):
            R = r + self.gamma * R
            returns.insert(0, R)
        return returns

    def train(self):
        episode_rewards = np.array([])
        with tqdm(range(self.num_episodes)) as pbar:
            for episode in pbar:
                state = self.env.reset()
                if not isinstance(state, np.ndarray):
                    state = np.array(state)
                episode_reward = 0

                log_probs = []
                values = []
                rewards = []

                for step in range(self.max_steps):
                    state_tensor = torch.FloatTensor(state).to(self.device)
                    action, log_prob = self.choose_action(state_tensor)
                    value = self.policy_net(state_tensor)[1]

                    next_state, reward, done, _ = self.env.step(action)
                    if not isinstance(next_state, np.ndarray):
                        next_state = np.array(next_state)

                    log_probs.append(log_prob.unsqueeze(0))
                    values.append(value.unsqueeze(0))
                    rewards.append(reward)

                    state = next_state
                    episode_reward += reward

                    if done:
                        break

                episode_rewards = np.append(episode_rewards, episode_reward)

                returns = self.compute_returns(rewards)
                returns = torch.FloatTensor(returns).to(self.device)


                if log_probs and values:
                    log_probs = torch.cat(log_probs).cpu()
                    values = torch.cat(values).cpu()
                    returns = returns.cpu()

                    advantage = returns - values

                    actor_loss = -(log_probs * advantage.detach()).mean()
                    critic_loss = self.critic_loss(values, returns)
                    loss = actor_loss + critic_loss

                    self.optimizer.zero_grad()
                    loss.backward()
                    self.optimizer.step()

                pbar.set_description(f"Episode {episode}, Reward: {np.mean(episode_rewards[-100:])}")

        self.env.close()
        return episode_rewards


Define the model and set hyperparameters.

In [7]:
env = gym.make('CartPole-v1')
num_episodes = 1000
max_steps = 500
lr = 1e-3
hidden_size = 256

a2c_model = A2CAgent(env, num_episodes=num_episodes, max_steps=max_steps, lr=lr, hidden_size=hidden_size)


Train the model.

In [8]:
rewards = a2c_model.train()

  0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:241: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([12])) that is different to the input size (torch.Size([12, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Episode 0, Reward: 12.0:   0%|          | 1/1000 [00:00<04:39,  3.57it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([24])) that is different to the input size (torch.Size([24, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, 

## Evaluation

Use the `choose_action` method of the trained agent to evaluate its performance.

In [16]:
import torch
import numpy as np
import imageio
from IPython.display import HTML
from base64 import b64encode
import gym

def show_video(path):
    mp4 = open(path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML("""
    <video width=400 controls>
          <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

env_name = 'CartPole-v1'

env = gym.make(env_name)

model = a2c_model

env.reset()

frames = []
episode_rewards = []

num_episodes = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for i in range(num_episodes):
    state = env.reset()
    state = np.array(state)
    episode_reward = 0

    done = False
    while not done:
        state_tensor = torch.FloatTensor(state).to(device)
        action, _ = model.choose_action(state_tensor)
        next_state, reward, done, _ = env.step(action)
        next_state = np.array(next_state)

        if i == 0:
            frame = env.render(mode="rgb_array")
            frames.append(frame)

        episode_reward += reward
        state = next_state

        if done:
            break

    episode_rewards.append(episode_reward)
    print(f"Episode Reward: {episode_reward}")

env.close()

episode_rewards = np.array(episode_rewards)
print(f"Average Reward over {num_episodes} episodes: {np.mean(episode_rewards)}")

# Save the video
video_path = './test.mp4'
imageio.mimsave(video_path, frames, fps=25)


Episode Reward: 500.0
Episode Reward: 500.0
Episode Reward: 500.0
Episode Reward: 500.0
Episode Reward: 500.0
Episode Reward: 486.0
Episode Reward: 316.0
Episode Reward: 368.0
Episode Reward: 483.0


Episode Reward: 500.0
Average Reward over 10 episodes: 465.3


In [17]:
show_video('./test.mp4')